In [2]:
import numpy as np
import pandas as pd
import gspread

In [3]:
client = gspread.service_account(filename="/Users/ahakso/Downloads/jobtracker.json")

url = 'https://docs.google.com/spreadsheets/d/1M__pvslmhMRkXCl-7DEPc0PzvKj6qlgfc8antAd9hgI/edit#gid=223128104'
sandbox_url = 'https://docs.google.com/spreadsheets/d/1Mab3WIIMxUuFdjzayu1kYBbeIf_-fsLI89vgx9GPKho/edit#gid=223128104'

wb = client.open_by_url(sandbox_url)

# Step 1: bring in the data & clean up


In [16]:
first_circuit = wb.get_worksheet_by_id(223128104)

In [24]:
colnames = first_circuit.get_values('A11:T11')[0]

In [31]:
raw = pd.DataFrame(first_circuit.get_values('A14:T500',),columns=colnames[:-1])

In [42]:
# list comprehension
munged_columns = [
    x.lower().replace(" ", "_").replace("/", "").replace("#", "no").replace("__", "_")
    for x in raw.columns
]
raw.columns = munged_columns
raw = raw.rename(columns={'squirt_boom': 'requires_squirt_boom'}).astype({'requires_squirt_boom': bool})

## Validate data

In [ ]:
gspread.Worksheet

 # Start to measure "distances" between jobs

## For now (maybe for always), never mix squirt & no squirt booms

In [53]:
print(len(raw.site_no.drop_duplicates()))
len(raw.site_no)

291


300

In [57]:
(raw.site_no=='283').sum()

2

In [59]:
raw.assign(unique_id=range(raw.shape[0]))

,status,site_no,address,full_address,contact,rmvl_rqust,work_description,nbw,owner_phone_comments,projected_hours,regular_,no_parks,flagging,requires_squirt_boom,merge,site_type,notes,trim_date,sign_off,unique_id
0,Done,1,525 N 125TH ST,"525 N 125TH ST Seattle, WA",DH,,ST CEDAR ON EVANSTON,,,0.75,X,,,False,,RegularCompleted,,6/14/23,H.M.R,0
1,Done,2,12245 EVANSTON AVE N,"12245 EVANSTON AVE N Seattle, WA",SH,,ST FIR & PINE / CLEAR POLE,,,1.00,X,,,False,,RegularCompleted,,6/15/23,H.M.R,1
2,Done,3,522 N 122ND ST,"522 N 122ND ST Seattle, WA",DH,,TT APPLE FOR NEUTRAL ON 122ND,,,0.75,X,,,False,,RegularCompleted,,6/15/23,H.M.R,2
3,In Process,4,523 N 122ND ST,"523 N 122ND ST Seattle, WA",DH,,ST CEDARS & FIR / TTS PINES / CLEAR POLE / WOR...,,,2.00,X,,,False,,RegularIn Process,,,,3
4,Not Started,5,12043 EVANSTON AVE N,"12043 EVANSTON AVE N Seattle, WA",DH,,TTS DEODAR AND MAPLE / CLEAR POLE,,,1.00,X,,,False,,Regular,,,,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0,0,Action Required,,AV,0.00,0.00,0.00,0.00,0.00,,,,False,0.00,,0.00,0.00,#DIV/0!,295
296,,,,,JW,0.00,0.00,0.00,0.00,0.00,,,,False,0.00,,0.00,0.00,#DIV/0!,296
297,,,,,NH,0.00,0.00,0.00,0.00,0.00,,,,False,0.00,,0.00,0.00,#DIV/0!,297
298,,,,,RO,0.00,0.00,0.00,0.00,0.00,,,,False,0.00,,0.00,0.00,#DIV/0!,298


In [55]:
raw.loc[lambda df: df.site_no=='283']

,status,site_no,address,full_address,contact,rmvl_rqust,work_description,nbw,owner_phone_comments,projected_hours,regular_,no_parks,flagging,requires_squirt_boom,merge,site_type,notes,trim_date,sign_off
282,Not Started,283,12751 EVANSTON AVE N,"12751 EVANSTON AVE N Seattle, WA",DH,,ST FIR / CLEAR POLE,,,1.00,X,,,False,,Regular,,,
294,423.75,283,Pending,,H.M.R,0.00,0.00,0.00,2.00,1.00,,,,False,3.00,,3.00,2.00,1.50


In [54]:
raw.site_no.value_counts()

       8
283    2
3      2
191    1
198    1
      ..
98     1
97     1
96     1
95     1
0      1
Name: site_no, Length: 291, dtype: int64

In [48]:
raw

,status,site_no,address,full_address,contact,rmvl_rqust,work_description,nbw,owner_phone_comments,projected_hours,regular_,no_parks,flagging,requires_squirt_boom,merge,site_type,notes,trim_date,sign_off
0,Done,1,525 N 125TH ST,"525 N 125TH ST Seattle, WA",DH,,ST CEDAR ON EVANSTON,,,0.75,X,,,False,,RegularCompleted,,6/14/23,H.M.R
1,Done,2,12245 EVANSTON AVE N,"12245 EVANSTON AVE N Seattle, WA",SH,,ST FIR & PINE / CLEAR POLE,,,1.00,X,,,False,,RegularCompleted,,6/15/23,H.M.R
2,Done,3,522 N 122ND ST,"522 N 122ND ST Seattle, WA",DH,,TT APPLE FOR NEUTRAL ON 122ND,,,0.75,X,,,False,,RegularCompleted,,6/15/23,H.M.R
3,In Process,4,523 N 122ND ST,"523 N 122ND ST Seattle, WA",DH,,ST CEDARS & FIR / TTS PINES / CLEAR POLE / WOR...,,,2.00,X,,,False,,RegularIn Process,,,
4,Not Started,5,12043 EVANSTON AVE N,"12043 EVANSTON AVE N Seattle, WA",DH,,TTS DEODAR AND MAPLE / CLEAR POLE,,,1.00,X,,,False,,Regular,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0,0,Action Required,,AV,0.00,0.00,0.00,0.00,0.00,,,,False,0.00,,0.00,0.00,#DIV/0!
296,,,,,JW,0.00,0.00,0.00,0.00,0.00,,,,False,0.00,,0.00,0.00,#DIV/0!
297,,,,,NH,0.00,0.00,0.00,0.00,0.00,,,,False,0.00,,0.00,0.00,#DIV/0!
298,,,,,RO,0.00,0.00,0.00,0.00,0.00,,,,False,0.00,,0.00,0.00,#DIV/0!


In [46]:
raw.requires_squirt_boom.mean()

0.0033333333333333335